In [1]:
import bokeh as bk
import bokeh.models as models
import bokeh.plotting
from bokeh.plotting import figure, show
import pandas as pd
import polars as pl

In [2]:
df = pd.DataFrame()
df['x'] = [1,2,3,4,5]
df['y'] = [5,4,3,2,1]

In [3]:
source = models.sources.ColumnDataSource(df)

In [4]:
fig = figure()
fig.scatter(source=source)

GlyphRenderer(id='p1038', ...)

In [29]:
show(fig)

In [7]:
pl.DataFrame(df)

x,y
i64,i64
1,5
2,4
3,3
4,2
5,1


In [4]:
x_length_grid_in_inches

16.125

In [18]:
from pathlib import Path
import pandas as pd
path = Path().home() / 'Python/data/tehaleh_wls.xlsx'
dfdd = pd.read_excel(path, sheet_name='WellDD').set_index('Date Time').apply(lambda col: pd.to_numeric(col, errors='coerce'))
dfdd.index.to_list()

WellHD


[Timestamp('1996-01-01 12:00:00'),
 Timestamp('1996-01-02 12:00:00'),
 Timestamp('1996-01-03 12:00:00'),
 Timestamp('1996-01-04 12:00:00'),
 Timestamp('1996-01-05 12:00:00'),
 Timestamp('1996-01-06 12:00:00'),
 Timestamp('1996-01-07 12:00:00'),
 Timestamp('1996-01-08 12:00:00'),
 Timestamp('1996-01-09 12:00:00'),
 Timestamp('1996-01-10 12:00:00'),
 Timestamp('1996-01-11 12:00:00'),
 Timestamp('1996-01-12 12:00:00'),
 Timestamp('1996-01-13 12:00:00'),
 Timestamp('1996-01-14 12:00:00'),
 Timestamp('1996-01-15 12:00:00'),
 Timestamp('1996-01-16 12:00:00'),
 Timestamp('1996-01-17 12:00:00'),
 Timestamp('1996-01-18 12:00:00'),
 Timestamp('1996-01-19 12:00:00'),
 Timestamp('1996-01-20 12:00:00'),
 Timestamp('1996-01-21 12:00:00'),
 Timestamp('1996-01-22 12:00:00'),
 Timestamp('1996-01-23 12:00:00'),
 Timestamp('1996-01-24 12:00:00'),
 Timestamp('1996-01-25 12:00:00'),
 Timestamp('1996-01-26 12:00:00'),
 Timestamp('1996-01-27 12:00:00'),
 Timestamp('1996-01-28 12:00:00'),
 Timestamp('1996-01-

In [20]:
import pandas as pd
from pathlib import Path
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
path = Path().home() / 'Python/data/tehaleh_wls.xlsx'
df_dd = pd.read_excel(path, sheet_name='WellDD').set_index('Date Time').apply(
    lambda column: pd.to_numeric(column, errors='coerce'))
df_dd = df_dd.drop(['Month', 'Year', 'Water Year'], axis='columns')
df_hd = pd.read_excel(path, sheet_name='WellHD').set_index('Date').apply(
    lambda column: pd.to_numeric(column, errors='coerce'))


WellHD
WellHD


In [38]:
tw6 = df_hd['TW-6'].dropna()
tw6.loc[:'10/2022'].min()

442.93999999999994

In [24]:
from figs._fig import Fig
fig = Fig()
for col in df_dd.columns:
    fig.add_scattergl(x=df_dd.index, y=df_dd[col], name=col)
fig.show()

NameError: name 'df_dd' is not defined

In [25]:
fig = Fig()
for col in df_hd.columns:
    df = df_hd[col].dropna()
    fig.add_scattergl(x=df.index, y=df, name=col)
fig.show()

NameError: name 'df_hd' is not defined

In [1]:
from _fig import Fig
import figs
import plotly.graph_objs as go
from pathlib import Path
import plotly.io as pio
fig = Fig()
import pandas as pd
import geopandas as gpd
import shapely as shp
shp_path = Path.home() / 'Python/data/explo/AESI_Explorations.shp'
wls_df = pd.read_excel(Path.home() / 'Python/data/Tehaleh_wls.xlsx', sheet_name='WellDD').set_index('Date Time')
pointshp = gpd.read_file(shp_path)
pointshp = pointshp.to_crs(4326)
wells_shp = pointshp[pointshp['ExploType']=='MW, EB-1W, TW, OBW']
wls_df = pd.read_excel(Path.home() / 'Python/data/Tehaleh_wls.xlsx', sheet_name='WellDD')

WellHD


In [7]:
map_fig = Fig()
y = wells_shp.geometry.y
x = wells_shp.geometry.x
map_center = {
    'lat': list(wells_shp.unary_union.centroid.coords)[0][1],
    'lon': list(wells_shp.unary_union.centroid.coords)[0][0]
}
wells_hover = {
    'Name': wells_shp['ExploName'].tolist(),
    'Proj #': wells_shp['AESI_ProjN'].tolist(),
    'Proj Name': wells_shp['AESI_Pro_2'].tolist()
}
custom_data, hover_template = figs.create_hover(wells_hover)
map_fig.add_scattermapbox(lat=y, lon=x, customdata=custom_data, hovertemplate=hover_template, marker_color='black',
                          text=wells_shp['ExploName'].tolist())
map_fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_center=map_center,
    mapbox_zoom=7
)
map_fig.show()
map_fig.write_json('fig.json')

In [5]:
wells_shp 

,id,AESI_ProjN,AESI_Pro_1,AESI_Pro_2,Phase,ExploType,ExploType_,ExploName,ExploDepth,ExploDate,...,Radiocarbo,Editor,Editor_res,EditDate,CreationDa,Creator,EditDate_1,Editor_1,wq,geometry
23,24.0,20130422,130422,Perrinville Creek,KHA,"MW, EB-1W, TW, OBW",MONITORING WELL,MW-1,87.0,2014/05/02 16:00:00.000,...,NaN,KLB,Karen Behm,2018/03/08 16:00:00.000,2023/01/31 16:59:15.279,nfink_AESI,2023/01/31 16:59:15.279,nfink_AESI,NaN,POINT (-122.34167 47.83094)
54,55.0,20130211,130211,Delridge,NaN,"MW, EB-1W, TW, OBW",MONITORING WELL,MW-1,NaN,NaN,...,NaN,KLB,Karen Behm,2018/03/08 16:00:00.000,2023/01/31 16:59:15.279,nfink_AESI,2023/01/31 16:59:15.279,nfink_AESI,NaN,POINT (-122.35606 47.53183)
55,56.0,20130211,130211,Delridge,NaN,"MW, EB-1W, TW, OBW",MONITORING WELL,MW-2,NaN,NaN,...,NaN,KLB,Karen Behm,2018/03/08 16:00:00.000,2023/01/31 16:59:15.279,nfink_AESI,2023/01/31 16:59:15.279,nfink_AESI,NaN,POINT (-122.35652 47.52569)
108,109.0,20130323,130323,Holy Family Parish,NaN,"MW, EB-1W, TW, OBW",MONITORING WELL,MW-1,NaN,NaN,...,NaN,KLB,Karen Behm,2018/03/08 16:00:00.000,2023/01/31 16:59:15.279,nfink_AESI,2023/01/31 16:59:15.279,nfink_AESI,NaN,POINT (-122.35899 47.51653)
111,112.0,20110220,110220,Barton Basin,KHA,"MW, EB-1W, TW, OBW",MONITORING WELL,MW-19,65.8,2011/10/19 16:00:00.000,...,NaN,KLB,Karen Behm,2019/11/05 16:00:00.000,2023/01/31 16:59:15.279,nfink_AESI,2023/01/31 16:59:15.279,nfink_AESI,NaN,POINT (-122.37278 47.53186)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20742,20743.0,20160066,160066,Well Installation and Testing,EHA,"MW, EB-1W, TW, OBW",MONITORING WELL,EW-2,423.0,2016/12/02 00:00:00.000,...,Yes,MPT,Michael Trop,2024/02/26 00:00:00.000,2024/02/26 17:36:44.746,mtrop_aesi,2024/02/26 17:50:15.810,mtrop_aesi,NaN,POINT (-122.65737 48.96475)
20743,20744.0,20160066,160066,Well Installation and Testing,E003,"MW, EB-1W, TW, OBW",MONITORING WELL,EW-3,600.0,2017/08/03 00:00:00.000,...,Yes,MPT,Michael Trop,2024/02/26 00:00:00.000,2024/02/26 17:58:14.244,mtrop_aesi,2024/02/26 18:00:05.174,mtrop_aesi,NaN,POINT (-122.67862 48.97206)
20787,20788.0,20120528,120528,New Wilson Pacific School,KEA,"MW, EB-1W, TW, OBW",MONITORING WELL,EB-9/MW-1,15.8,2013/02/01 00:00:00.000,...,NaN,MPT,Michael Trop,2024/02/26 00:00:00.000,2024/02/26 22:43:58.530,mtrop_aesi,2024/02/26 23:02:32.724,mtrop_aesi,NaN,POINT (-122.33926 47.69468)
20803,20804.0,20150005,150005,Park 77,EEA,"MW, EB-1W, TW, OBW",MONITORING WELL,MW-1,36.0,2015/09/21 00:00:00.000,...,Yes,MPT,Michael Trop,2024/02/26 00:00:00.000,2024/02/26 23:43:08.168,mtrop_aesi,2024/02/26 23:47:06.220,mtrop_aesi,NaN,POINT (-122.12640 48.17892)


In [3]:
f = Fig()
f.show()

In [5]:
import plotly
import itertools
cycle = itertools.cycle(plotly.colors.DEFAULT_PLOTLY_COLORS)
next(cycle)

'rgb(31, 119, 180)'

In [25]:
next(cycle)

'rgb(31, 119, 180)'

In [3]:
import figs
figs.Fig

['__new__',
 '__getattribute__',
 '__get__',
 '__set__',
 '__delete__',
 '__init__',
 'getter',
 'setter',
 'deleter',
 '__set_name__',
 'fget',
 'fset',
 'fdel',
 '__doc__',
 '__isabstractmethod__',
 '__repr__',
 '__hash__',
 '__str__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce_ex__',
 '__reduce__',
 '__getstate__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']